In [14]:
import warnings
warnings.simplefilter(action='ignore', category=Warning)

import pandas as pd

import nltk
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords
#from nltk.tokenize import sent_tokenize, word_tokenize

from nltk.stem.snowball import SnowballStemmer

from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *

In [2]:
replace_str1 = '<span class="css-1egxyvc" data-font-weight="bold">croissant</span>'
replace_str2 = '<span class="css-1egxyvc" data-font-weight="bold">croissants</span>'

In [3]:
reviews = []

with open("../archive/reviews.txt", "r") as the_file:
    while True:
        line = the_file.readline()
        if line == "":
            break
        elif len(line) > 3: # \n
            line = line.replace(replace_str1, "croissant")
            line = line.replace(replace_str2, "croissant")
            reviews.append(line)

print(len(reviews))

90


In [4]:
print(reviews[1:4])

['lost larson bakery : These are the best cinnamon rolls ever. The croissant are also delicious and the service is very efficient”\n', 'bang bang pie and biscuits : reminded me of a croissant) and light. The actual filling was very good and you can clearly taste”\n', 'good ambler : minutes. - Ham and cheese croissant: croissant was super flaky and the sesame seeds were a nice touch”\n']


In [5]:
# I like this version better but pandas words differently, so later we conform
dict_reviews = {review.split(' : ')[0]: review.split(' : ')[1] for review in reviews}

names = list(dict_reviews.keys())
words = list(dict_reviews.values())
index = ['idx'+str(idx) for idx in range(0, len(names))]
dict_for_pd = {'index': index, 'BakeryName': names, 'Review': words}

df = pd.DataFrame.from_dict(dict_for_pd)
df.set_index("index", inplace = True)
df.head()

,BakeryName,Review
index,,
idx0,hendrickx belgian bread crafter,The almond chocolate croissant and french coun...
idx1,lost larson bakery,These are the best cinnamon rolls ever. The cr...
idx2,bang bang pie and biscuits,reminded me of a croissant) and light. The act...
idx3,good ambler,minutes. - Ham and cheese croissant: croissant...
idx4,p%C3%A2tisserie coralie,is a croissant desert. Search no more. Very fr...


In [6]:
# some peering
pd.options.display.max_colwidth = None
df[df['Review'].str.contains("flak")]

,BakeryName,Review
index,,
idx3,good ambler,minutes. - Ham and cheese croissant: croissant was super flaky and the sesame seeds were a nice touch”\n
idx16,hoosier mama pie company,and cream admixture and a flaky buttery crust almost like that of a croissant. Indecisive and gluttonous as”\n
idx33,caffe umbria,"are the highlight of the show, you absolutely have to get a croissant - it was so flaky and buttery and the muffin was also moist and had a hint of cinnamon!!!”\n"
idx39,"abc bakery and deli norridge?hrid=SDxAypM6HgjzV47KHlp0TQ&amp;osq=croissant"">more</a></span></p","of the seductive sweets. I chose a nutella croissant! Fluffy, flaky, sticky hazelnut spread glory. Yes”\n"


In [7]:
# hoosier mama is not a review on a croissant
# not a huge nutella fan, it's just sugar. too easy
# caffe umbria and good ambler are the winners based on this.
# but let's do more with these reviews
# while we r here. 

In [8]:
df[df['Review'].str.contains("artisan")]

,BakeryName,Review
index,,


In [9]:
# tokenizing, filtering stop words, stemming, and lower casing

stop_words = set(stopwords.words("english"))
stemmer = SnowballStemmer('english')

In [10]:
df['Review_Prepped'] = df.Review.apply(lambda xyz: 
                                       [stemmer.stem(word.lower()) 
                                        for word in wordpunct_tokenize(xyz) 
                                        if word.lower() not in stop_words])

In [11]:
df.head()

,BakeryName,Review,Review_Prepped
index,,,
idx0,hendrickx belgian bread crafter,The almond chocolate croissant and french country bread were quite authentic and wonderfully rustic.”\n,"[almond, chocol, croissant, french, countri, bread, quit, authent, wonder, rustic, .”]"
idx1,lost larson bakery,These are the best cinnamon rolls ever. The croissant are also delicious and the service is very efficient”\n,"[best, cinnamon, roll, ever, ., croissant, also, delici, servic, effici, ”]"
idx2,bang bang pie and biscuits,reminded me of a croissant) and light. The actual filling was very good and you can clearly taste”\n,"[remind, croissant, ), light, ., actual, fill, good, clear, tast, ”]"
idx3,good ambler,minutes. - Ham and cheese croissant: croissant was super flaky and the sesame seeds were a nice touch”\n,"[minut, ., -, ham, chees, croissant, :, croissant, super, flaki, sesam, seed, nice, touch, ”]"
idx4,p%C3%A2tisserie coralie,"is a croissant desert. Search no more. Very friendly staff, great tea and coffee. A truly welcome addition to the neighborhood. Best of luck.”\n","[croissant, desert, ., search, ., friend, staff, ,, great, tea, coffe, ., truli, welcom, addit, neighborhood, ., best, luck, .”]"


In [12]:
# semantic analysis